## **Jobsheet 10 | Pertemuan 10**
## **Recurent Neural Network (RNN)**

### **Tugas**

Prosedur pelatihan pada praktikum 2 merupakan prosedur sederhana, yang tidak memberi Anda banyak kendali. Model ini menggunakan "teacher-forcing" yang mencegah prediksi buruk diumpankan kembali ke model, sehingga model tidak pernah belajar untuk pulih dari kesalahan. Jadi, setelah Anda melihat cara menjalankan model secara manual, selanjutnya Anda akan mengimplementasikan custom loop pelatihan. Hal ini memberikan titik awal jika, misalnya, Anda ingin menerapkan pembelajaran kurikulum untuk membantu menstabilkan keluaran open-loop model. Bagian terpenting dari loop pelatihan khusus adalah fungsi langkah pelatihan.<p>
Gunakan tf.GradientTape untuk men track nilai gradient. Anda dapat mempelajari lebih lanjut tentang pendekatan ini dengan membaca eager execution guide.<p>

Prosedurnya adalah "<p>
1. Jalankan Model dan hitung loss dengan tf.GradientTape.
2. Hitung update dan terapkan pada model dengan optimizer


### **Jawab**

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

melakukan download Dataset Shakespeare

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


Load Data

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [8]:
# Print unique characters
for char in vocab:
    print(char, end=' ')


   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z 

olah Teks

Vectorize Teks

Sebelum training, Anda perlu mengonversi string menjadi representasi numerik. tf.keras.layers.StringLookup dapat mengubah setiap karakter menjadi ID numerik. Caranya adalah teks akan dipecah menjadi token terlebih dahulu.

In [9]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

kemudian buat tf.keras.layers.StringLookup layer:

In [11]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [12]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [13]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [14]:

chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [15]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [16]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

Prediksi

Membuat Training Set dan Target

In [17]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [18]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [19]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [20]:
seq_length = 100

Metode batch memungkinkan kita dengan mudah mengonversi karakter individual ini menjadi urutan ukuran yang diinginkan.

In [21]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


akan lebih mudah untuk melihat apa yang dilakukan jika Anda menggabungkan token kembali menjadi string:

In [22]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


Untuk pelatihan, Anda memerlukan kumpulan data pasangan (input, label). Dimana input dan label merupakan urutan. Pada setiap langkah waktu, inputnya adalah karakter saat ini dan labelnya adalah karakter berikutnya. Berikut adalah fungsi yang mengambil urutan sebagai masukan, menduplikasi, dan menggesernya untuk menyelaraskan masukan dan label untuk setiap langkah waktu:

In [23]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [24]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [25]:
dataset = sequences.map(split_input_target)

In [26]:
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Membuat Batch Training

Anda menggunakan tf.data untuk membagi teks menjadi sequence yang dapat diatur. Namun sebelum memasukkan data ini ke dalam model, Anda perlu mengacak data dan mengemasnya ke dalam batch.

In [27]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

Membuat Model

In [28]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [29]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [30]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Uji Model

In [31]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [32]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
sampled_indices = tf.random.categorical(example_batch_predictions[0],num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [34]:
sampled_indices

array([ 0, 42, 52, 52, 49, 49, 49, 20, 55, 62, 39,  5, 35, 48, 24, 64, 13,
       65, 49,  7,  2, 29, 20, 39, 12,  6,  4, 59, 30, 62, 49, 34, 37, 21,
       60, 35, 17,  5,  5, 33, 53,  8, 57,  8, 26, 48, 27, 48, 34,  0, 61,
       58, 35, 39, 34, 26, 26, 13, 34, 57,  9, 19, 53, 33,  6, 44, 51,  5,
       60, 17, 61, 22, 15, 61,  8, 14, 61, 23, 63, 32, 50, 31, 35, 34, 51,
       34, 43, 10,  1,  0, 44, 20, 50, 17, 39, 44, 61, 45, 48, 15])

Dekode kode berikut untuk melihat teks yang diprediksi oleh model tidak terlatih ini:

In [35]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' counsel to his unstaid youth?\n\nDUKE OF YORK:\nVex not yourself, nor strive not with your breath;\nFor'

Next Char Predictions:
 b"[UNK]cmmjjjGpwZ&ViKy?zj, PGZ;'$tQwjUXHuVD&&Tn-r-MiNiU[UNK]vsVZUMM?Ur.FnT'el&uDvIBv-AvJxSkRVUlUd3\n[UNK]eGkDZevfiB"


### **Train Model**

Tambahan optimizer dan fungsi loss<p>

loss function tf.keras.losses.sparse_categorical_crossentropy standar berfungsi dalam kasus ini karena diterapkan di seluruh dimensi terakhir prediksi. Karena model Anda mengembalikan logits, Anda perlu mengatur flag from_logits.

In [36]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [37]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.188585, shape=(), dtype=float32)


Model yang baru diinisialisasi tidak boleh terlalu yakin dengan dirinya sendiri, semua log keluaran harus memiliki besaran yang sama. Untuk mengonfirmasi hal ini, Anda dapat memeriksa bahwa eksponensial dari loss rata-rata harus kira-kira sama dengan ukuran kosakata. Loss yang jauh lebih tinggi berarti model tersebut yakin akan jawaban yang salah, dan memiliki inisialisasi yang buruk:

In [38]:
tf.exp(example_batch_mean_loss).numpy()

65.92942

Konfigurasikan prosedur pelatihan menggunakan metode tf.keras.Model.compile. Gunakan tf.keras.optimizers.Adam dengan argumen default dan fungsi loss.

In [39]:
model.compile(optimizer='adam', loss=loss)

Konfigurasi Checkpoints<p>
Gunakan tf.keras.callbacks.ModelCheckpoint untuk memastikan bahwa checkpoint disimpan selama pelatihan:

In [40]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Lakukan Proses Training

In [41]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 15s 55ms/step - loss: 2.7392
Epoch 2/10
172/172 [==============================] - 12s 55ms/step - loss: 2.0058
Epoch 3/10
172/172 [==============================] - 12s 57ms/step - loss: 1.7215
Epoch 4/10
172/172 [==============================] - 12s 59ms/step - loss: 1.5549
Epoch 5/10
172/172 [==============================] - 11s 57ms/step - loss: 1.4552
Epoch 6/10
172/172 [==============================] - 12s 58ms/step - loss: 1.3860
Epoch 7/10
172/172 [==============================] - 13s 55ms/step - loss: 1.3329
Epoch 8/10
172/172 [==============================] - 11s 55ms/step - loss: 1.2884
Epoch 9/10
172/172 [==============================] - 11s 56ms/step - loss: 1.2486
Epoch 10/10
172/172 [==============================] - 11s 55ms/step - loss: 1.2092


Generate Teks<p>

Berikut ini membuat prediksi satu langkah:

In [42]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [43]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [44]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Now. God send, the senater:
if him the keporty should to kill the figure in the book.

DUKE VINCENTIO:
What, is my wift as the hand: what shall shall stand upon my livery?

ROMEO:
Then he knows, nor will now, saddle sife have
perfective; Belaps. Thereof is the centrement of his
Against myself I say,
With her hence till no other, so, can ster a
last! it assurity,
I warrant him. Be entreat day's blunch.

CAPULET:
What is the mast embrace:
or, bear it. For I hear, beseech yourselves, and
Burks, traitor, to Excoup 's, had no money and death.
Poor Margaret, man, a curse is head into my wife.

QUEEN ELIZABETH:
what chose the issue of this praise strokes murdersh in our morning
Commends' secretive formins, your advice here,
As enemies, desperate worth themself,
Have no increemity an assuran
To set forty on me. the word of pasting
By with your ammistacence, in Rome.

MENENIUS:
That is such brools is so.

Clown:
Here's a new-better news.

DUKE OF AUMERLE:
And-done win dicord, I will not 

In [45]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nO, she had shaked friends. Both, being met's well,\nWith ungreen wench obstance, Signior Husbany!\n\nFiest Catesby:\nPrey thou, this sudden dows,\nCannot subsurvey. Loth he go back,\nTo time hath cares for the poilty:\nBeing not decey.'\n\nKING EDWARD IV:\nAlack the such a ligh, and cheer to speak with splece a luin;\ncoadle, by night. I warrant, sir, brister.\n\nMENENIUS:\nO, there's me no more.\n\nNurse:\nAn if I cannot tell him;\nThat dark'd up ourselves, want thou weep my cheek\nWhen I prod strength whose dead God, although with hone,\nwe would not be behelding his heaven's,\nThat, canourdy in the time,\nWhich, tender women knows for peers\nNo remised him from his tongue?\n\nFRIAR JAENCA:\nFor wail in the substace of the worst,\nWho shows perpected his approfected men.\n\nANTONIO:\nThe widow his heeds.\n\nFirst Citizen:\nAy, let the prince shall not\nHow can I not, sit now?\n\nCORIOLANUS:\nNo, number\nhere all make him mad so blows. But when,\nWith Richard Cas

Ekspor Model Generator

In [46]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [47]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
O love, part meant in parliamenes,
Your affects of albow and to me; he be seen'.

GREY:
She speaks,


-----
Tugas

In [48]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
    inputs, labels = inputs
    with tf.GradientTape() as tape:
      predictions = self(inputs, training=True)
      loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [49]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [50]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [51]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 16s 57ms/step - loss: 2.7309


In [52]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
  start = time.time()

  mean.reset_states()
  for (batch_n, (inp, target)) in enumerate(dataset):
    logs = model.train_step([inp, target])
    mean.update_state(logs['loss'])

    if batch_n % 50 == 0:
      template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
      print(template)

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print()
  print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
  print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
  print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1922
Epoch 1 Batch 50 Loss 2.0653
Epoch 1 Batch 100 Loss 1.9274
Epoch 1 Batch 150 Loss 1.8461

Epoch 1 Loss: 1.9951
Time taken for 1 epoch 12.38 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.7827
Epoch 2 Batch 50 Loss 1.7092
Epoch 2 Batch 100 Loss 1.6914
Epoch 2 Batch 150 Loss 1.6794

Epoch 2 Loss: 1.7168
Time taken for 1 epoch 10.86 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5758
Epoch 3 Batch 50 Loss 1.5872
Epoch 3 Batch 100 Loss 1.5039
Epoch 3 Batch 150 Loss 1.5213

Epoch 3 Loss: 1.5561
Time taken for 1 epoch 10.72 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4540
Epoch 4 Batch 50 Loss 1.4406
Epoch 4 Batch 100 Loss 1.4865
Epoch 4 Batch 150 Loss 1.4472

Epoch 4 Loss: 1.4572
Time taken for 1 epoch 12.00 sec
_____________________________________________________________________

### **Jawaban Kesimpulan**

Perbedaan antara kode tugas dengan praktikum 2 terletak pada prosedur pelatihan. Pada praktikum 2 menggunakan pendekatan pelatihan yang lebih sederhana dan umum digunakan, dengan model.fit. Sedangkan kode pada tugas menggambarkan pendekatan pelatihan yang lebih spesifik dan kompleks, yang dilakukan beberapa kustomisasi.<p>

Dalam pendekatan ini, mendefinisikan metode train_step dalam model turunan yang mengatur pelatihan pada tingkat batch. Secara eksplisit dilakukan perhitungan loss, gradien, dan menerapkan pembaruan bobot model dengan apply_gradients, serta menggunakan objek tf.metrics.Mean untuk menghitung rata-rata loss selama pelatihan. Pendekatan ini memberikan lebih banyak kontrol dan fleksibilitas dalam pengaturan pelatihan model.